In [7]:
!pip install en_core_web_md-3.5.0.tar.gz


Processing c:\users\wajiz.pk\desktop\ai intern task\chatbot\en_core_web_md-3.5.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached spacy-3.5.4-cp310-cp310-win_amd64.whl.metadata (25 kB)
  Using cached thinc-8.1.12-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-1.10.21-cp310-cp310-win_amd64.whl.metadata (155 kB)
  Using cached blis-0.7.11-cp310-cp310-win_amd64.whl.metadata (7.6 kB)
Using cached spacy-3.5.4-cp310-cp310-win_amd64.whl (12.2 MB)
Using cached pydantic-1.10.21-cp310-cp310-win_amd64.whl (2.3 MB)
Using cached thinc-8.1.12-cp310-cp310-win_amd64.whl (1.5 MB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
Using cached blis-0.7.11-cp310-cp310-win_amd64.whl (6.6 MB)
  Created wheel for en_core_web_md: filename=en_core_web_md-3.5.0-py3-none-any.whl size=42778850 sha256=b98c22d7f89e9fe8cb4a791dacec979d5c52de82a3866b7ca83f4405bb01

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.7.0 requires pydantic>=2.0, but you have pydantic 1.10.21 which is incompatible.
gradio 5.7.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
langchain 0.3.9 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
langchain-core 0.3.21 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
pydantic-settings 2.6.1 requires pydantic>=2.7.0, but you have pydantic 1.10.21 which is incompatible.


In [ ]:
!pip install ipykernel

: 

In [ ]:
!pip install -r requirements.txt


: 

Load spaCy and Preprocess FAQs

In [5]:
import spacy
import json

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Use a medium model for better accuracy

# Load FAQs
with open("faqs.json", "r") as f:
    faqs = json.load(f)

# Preprocess FAQs for semantic similarity
def preprocess_faqs(faqs):
    return [(faq['question'], nlp(faq['question'])) for faq in faqs]

faq_data = preprocess_faqs(faqs)


PydanticImportError: `pydantic:ConstrainedStr` has been removed in V2.

For further information visit https://errors.pydantic.dev/2.10/u/import-error

Create a Function for Matching Questions

In [10]:
def get_best_match(user_query, faq_data, threshold=0.7):
    query_doc = nlp(user_query)
    best_match = None
    best_score = 0

    for question, question_doc in faq_data:
        similarity = query_doc.similarity(question_doc)
        if similarity > best_score:
            best_match = question
            best_score = similarity

    if best_score > threshold:
        return best_match
    else:
        return "Sorry, I couldn't find an answer to your question."


Integrate with gradio for a Simple Web Chatbot

In [17]:
import json
from difflib import get_close_matches
import gradio as gr

# Load FAQs from the JSON file
try:
    with open("faqs.json", "r") as f:
        faq_data = json.load(f)
except Exception as e:
    faq_data = []
    print(f"Error loading FAQs: {e}")

# Function to find the best matching question
def get_best_match(user_input, faq_data):
    questions = [faq["question"] for faq in faq_data]
    matches = get_close_matches(user_input, questions, n=1, cutoff=0.6)
    return matches[0] if matches else None

# Chatbot function
def chatbot(user_input):
    if not user_input.strip():
        return "Please ask a question!"
    
    best_question = get_best_match(user_input, faq_data)
    if best_question:
        for faq in faq_data:
            if faq["question"] == best_question:
                return faq["answer"]
    return "Sorry, I couldn't find an answer to your question."

# Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Ask a question"),
    outputs=gr.Textbox(label="Answer"),
    title="FAQ Chatbot",
    description="This chatbot answers FAQs about popular smartphones in 2025 in Pakistan. Type your question below!",
)

# Launch the Gradio interface
if __name__ == "__main__":
    iface.launch()



KeyboardInterrupt

